# Cross Subject Classification with Inception CNN

# Importing Packages

In [ ]:
!pip install braindecode
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.5 MB/s eta 0:00:00


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import braindecode
import mne

In [ ]:
from scipy.io import loadmat
import scipy
import sklearn
import numpy as np
import pandas as pd
import glob
from mne.decoding import CSP
import os

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, LeaveOneGroupOut, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

In [ ]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [ ]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [ ]:
n_jobs = -1  # for multicore parallel processing, set it to 1 if cause memory issues, for full utilization set to -1

# Data Loading

## Kaggle

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
          pass

In [ ]:
folder_path = "/kaggle/input/thumbmovement/Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow-master"

## Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data"

## Split 10 Subjects to 8 Training and 2 Testing


In [ ]:
all_files   = glob.glob(folder_path + '/*T.mat')
len(all_files)     # if  return zero,then no file is loaded

10

In [ ]:
all_files

['/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P10T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P09T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P03T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P02T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P04T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P01T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P05T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P07T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P06T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P08T.mat']

In [ ]:
train_subjects = all_files[0:8]
print(len(train_subjects))
test_subjects = all_files[8:]
print(len(test_subjects))

8
2


In [ ]:
train_subjects

['/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P10T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P09T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P03T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P02T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P04T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P01T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P05T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P07T.mat']

In [ ]:
test_subjects

['/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P06T.mat',
 '/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data/parsed_P08T.mat']

# Data Processing
Data Loading with Epochs & Band Pass Filtering & cutting  [0.5, 4.5] sec and 2sec window with 1 sec stride

## Epoch conversion

In [ ]:
def get_mne_epochs_complete(files_paths, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    similar to get_mne_epochs, just appends data from all relevant files together to give a single
    epoch object
    '''
    eeg_data = []
    for filepath in files_paths:
        mat_data = loadmat(filepath)
        eeg_data.extend(mat_data['RawEEGData'])

    idx_start = fs*t_start      # fs*ts
    eeg_data = np.array(eeg_data)
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 0, 'right-hand': 1} # pytorch expects labels in [0, n_classes-1]
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None) # required be ICA, (7-30 Hz) later
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction

    if mode == 'train': # this in only applicable for training data
        labels = []
        for filepath in files_paths:
            mat_data = loadmat(filepath)
            labels.extend(mat_data['Labels'].ravel() - 1)
        epochs.event_id = event_id
        epochs.events[:,2] = labels
    return epochs

In [ ]:
# loading data and filtering from 7 to 32 HZ
train_subjects_epochs_all = get_mne_epochs_complete(train_subjects).filter(7,32) # for all training subjects
test_subjects_epochs_all = get_mne_epochs_complete(test_subjects).filter(7,32) # for all training subjects

In [ ]:
train_subjects_epochs_all.get_data().shape

(640, 12, 3072)

In [ ]:
train_subjects_epochs_all

Number of events,640
Events,left-hand: 320right-hand: 320
Time range,-1.000 – 4.998 s
Baseline,-0.250 – 0.000 s


In [ ]:
# Croping Data from 0.5 to 4.5 (4 seconds * 512)
# train_subjects_epochs_all = train_epochs_all.crop(tmin=0.5, tmax=4.5, include_tmax=False)
# train_subjects_epochs_all = test_epochs_all.crop(tmin=0.5, tmax=4.5, include_tmax=False)

In [ ]:
train_subjects_epochs_all.get_data().shape

(640, 12, 3072)

In [ ]:
# # Define the indices for slicing (Train , Test)
# train_indices = []
# test_indices = []

# # Calculate the indices for training epochs (0 to 49 and 70 to 79 for each subject)
# for i in range(8):
#     train_indices.extend(list(range(i * 80, i * 80 + 50)))
#     train_indices.extend(list(range(i * 80 + 70, i * 80 + 80)))

# # Calculate the indices for testing epochs (50 to 69 for each subject)
# for i in range(8):
#     test_indices.extend(list(range(i * 80 + 50, i * 80 + 70)))

In [ ]:
# Define the indices for slicing  (Train , Test, Validation)
train_indices = []
test_indices = []
val_indices = []

# Calculate the indices for training epochs (0 to 39 for each subject)
for i in range(8):
    train_indices.extend(list(range(i * 80, i * 80 + 40)))

# Calculate the indices for testing epochs (50 to 69 for each subject)
for i in range(8):
    test_indices.extend(list(range(i * 80 + 50, i * 80 + 70)))

# Calculate the indices for validation epochs (40 to 49 and 70 to 79 for each subject)
for i in range(8):
    val_indices.extend(list(range(i * 80 + 40, i * 80 + 50)))
    val_indices.extend(list(range(i * 80 + 70, i * 80 + 80)))

In [ ]:
train_subjects_epochs_train = train_subjects_epochs_all[train_indices]
train_subjects_epochs_test  = train_subjects_epochs_all[test_indices]
train_subjects_epochs_val  = train_subjects_epochs_all[val_indices]

In [ ]:
train_subjects_epochs_train

Number of events,320
Events,left-hand: 160right-hand: 160
Time range,-1.000 – 4.998 s
Baseline,-0.250 – 0.000 s


In [ ]:
train_subjects_epochs_test

Number of events,160
Events,left-hand: 80right-hand: 80
Time range,-1.000 – 4.998 s
Baseline,-0.250 – 0.000 s


In [ ]:
train_subjects_epochs_val

Number of events,160
Events,left-hand: 80right-hand: 80
Time range,-1.000 – 4.998 s
Baseline,-0.250 – 0.000 s


In [ ]:
# train_subjects_epochs_train
# train_subjects_epochs_test
# train_subjects_epochs_val
# test_subjects_epochs_all

In [ ]:
# train_subjects_epochs = train_epochs_all.copy()
# test_subjects_epochs = test_epochs_all.copy()
# train_subjects_data, train_subjects_labels = train_subjects_epochs.get_data(), train_subjects_epochs.events[:,-1]
# test_subjects_data, test_subjects_labels = test_subjects_epochs.get_data(), test_subjects_epochs.events[:,-1]
# print('Shape of EEG Train Subjects Data: ', train_subjects_data.shape, '\t Shape of Labels: ', train_subjects_labels.shape)
# print('Shape of EEG Test Subjects Data: ', test_subjects_data.shape, '\t Shape of Labels: ', test_subjects_labels.shape)

## Windows Dataset Conversion
Type : [BaseConcatDataset](https://braindecode.org/stable/generated/braindecode.datasets.BaseConcatDataset.html#braindecode.datasets.BaseConcatDataset)

Taking window size of 2 seconds with stride 1 second

In [ ]:
from braindecode.datautil import create_from_mne_epochs

# convert epochs to braindecode compatible datastructure
# 2sec windows with 0.125 sec stride
window_size = 1024 #1024 #1024 #50 # 3072
window_stride = 512 #512 #256 # 50

In [ ]:
# train_windows_datasets = create_from_mne_epochs(
#             [train_epochs_all], # expects list of epochs
#             window_size_samples = window_size, window_stride_samples = window_stride,
#             drop_last_window = False )

# test_windows_datasets = create_from_mne_epochs(
#             [test_epochs_all], # expects list of epochs
#             window_size_samples = window_size, window_stride_samples = window_stride,
#             drop_last_window = False )

In [ ]:
train_subjects_windows_datasets_train = create_from_mne_epochs(
            [train_subjects_epochs_train], # expects list of epochs
            window_size_samples = window_size, window_stride_samples = window_stride,
            drop_last_window = False )

train_subjects_windows_datasets_test = create_from_mne_epochs(
            [train_subjects_epochs_test], # expects list of epochs
            window_size_samples = window_size, window_stride_samples = window_stride,
            drop_last_window = False )

train_subjects_windows_datasets_val = create_from_mne_epochs(
            [train_subjects_epochs_val], # expects list of epochs
            window_size_samples = window_size, window_stride_samples = window_stride,
            drop_last_window = False )

test_subjects_windows_datasets_all = create_from_mne_epochs(
            [test_subjects_epochs_all], # expects list of epochs
            window_size_samples = window_size, window_stride_samples = window_stride,
            drop_last_window = False )

In [ ]:
# train_subjects_windows_datasets_train
# train_subjects_windows_datasets_test
# train_subjects_windows_datasets_val
# test_subjects_windows_datasets_all

In [ ]:
def get_windows_datasets_labels(windows_dataset):
    labels = []
    for i in range(len(windows_dataset.datasets)):
        labels.extend(windows_dataset.datasets[i].y)
    return np.array(labels)

In [ ]:
# train_windows_description = pd.DataFrame(data=get_windows_datasets_labels(train_windows_datasets),columns=['labels'])
# test_windows_datasets_description = pd.DataFrame(data=get_windows_datasets_labels(test_windows_datasets),columns=['labels'])

In [ ]:
train_subjects_train_labels= get_windows_datasets_labels(train_subjects_windows_datasets_train)
train_subjects_test_labels= get_windows_datasets_labels(train_subjects_windows_datasets_test)
train_subjects_val_labels= get_windows_datasets_labels(train_subjects_windows_datasets_val)
test_subjects_labels= get_windows_datasets_labels(test_subjects_windows_datasets_all)

In [ ]:
print(train_subjects_train_labels)
print(len(train_subjects_train_labels))

[1 1 1 ... 1 1 1]
1600


In [ ]:
print("Total Windows in a train Dataset: ", len(train_subjects_train_labels))
print("Total Windows in a test Dataset: ", len(train_subjects_test_labels))
print("Total Windows in a val Dataset: ", len(train_subjects_val_labels))
print("Total Windows in a out group test Dataset: ", len(test_subjects_labels))

Total Windows in a train Dataset:  1600
Total Windows in a test Dataset:  800
Total Windows in a val Dataset:  800
Total Windows in a out group test Dataset:  800


In [ ]:
# train_windows_description

In [ ]:
# train_windows_datasets.set_description = train_windows_description

In [ ]:
# train_windows_datasets.set_description.labels[500:]

In [ ]:
# print("Total Windows in a whole Dataset: ", len(train_windows_description))

## Preprocessing

In [ ]:
from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
)

In [ ]:
low_cut_hz = 8.  # low cut frequency for filtering
high_cut_hz = 32.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

In [ ]:
# def custom_exp_moving_std_fn(epochs, factor_new=factor_new, init_block_size=init_block_size):
#     data = epochs.get_data()
#     for i in range(len(data)):
#         epochs._data[i] = exponential_moving_standardize(data[i],
#                         factor_new=factor_new, init_block_size=init_block_size)
#     return epochs

In [ ]:
preprocessors = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    # Preprocessor(
    #     # fn=custom_exp_moving_std_fn,  # Exponential moving standardization
    #     factor_new=factor_new,
    #     init_block_size=init_block_size,
    # ),
]

In [ ]:
preprocess

In [ ]:
# preprocess(windows_datasets, preprocessors)
train_subjects_windows_datasets_train = preprocess(train_subjects_windows_datasets_train, preprocessors, n_jobs=-1)

In [ ]:
train_subjects_windows_datasets_test = preprocess(train_subjects_windows_datasets_test, preprocessors, n_jobs=-1)

In [ ]:
train_subjects_windows_datasets_val = preprocess(train_subjects_windows_datasets_val, preprocessors, n_jobs=-1)

In [ ]:
test_subjects_windows_datasets_all = preprocess(test_subjects_windows_datasets_all, preprocessors, n_jobs=-1)

## Convertion from window Dataset to 3D Numpy Array

### Window Dataset Exploration

In [ ]:
print(len(train_windows_datasets[0]))
train_windows_datasets[0]

3


In [ ]:
print(len(train_windows_datasets[0][0]))
print(train_windows_datasets[0][0].shape)
#12 Channel * 1024 Sample (2 Seconds * 512)
# train_windows_datasets[0][0]

12
(12, 1024)


In [ ]:
type(train_windows_datasets[1][0])

numpy.ndarray

In [ ]:
print(len(train_windows_datasets[0][0][0]))
print(train_windows_datasets[0][0][0].shape)
train_windows_datasets[0][0][0]

1024
(1024,)


array([ -4028139.2,  -3766319. ,  -3094056.5, ..., -20674458. ,
       -18828416. , -15577379. ], dtype=float32)

In [ ]:
print(len(train_windows_datasets))
print(len(test_windows_datasets))
#1920 Window

1920
480


In [ ]:
type(train_windows_datasets)

braindecode.datasets.base.BaseConcatDataset

In [ ]:
# len(windows_datasets.set_description[0])

### Conversion from window to list

In [ ]:
# if 408 % 240 > 168:
#   print("YES")

In [ ]:
train_subjects_windows_datasets_train_list = []
for window in train_subjects_windows_datasets_train:
  train_subjects_windows_datasets_train_list.append(window[0])
print(len(train_subjects_windows_datasets_train_list))


1600


In [ ]:
train_subjects_windows_datasets_test_list = []
for window in train_subjects_windows_datasets_test:
  train_subjects_windows_datasets_test_list.append(window[0])
print(len(train_subjects_windows_datasets_test_list))

800


In [ ]:
train_subjects_windows_datasets_val_list = []
for window in train_subjects_windows_datasets_val:
  train_subjects_windows_datasets_val_list.append(window[0])
print(len(train_subjects_windows_datasets_val_list))

800


In [ ]:
test_subjects_windows_datasets_all_list = []
for window in test_subjects_windows_datasets_all:
  test_subjects_windows_datasets_all_list.append(window[0])
print(len(test_subjects_windows_datasets_all_list))

800


In [ ]:
# train_subjects_windows_datasets_train_list
# train_subjects_windows_datasets_test_list
# train_subjects_windows_datasets_val_list
# test_subjects_windows_datasets_all_list

In [ ]:
# train = []
# test = []
# train_labels = []
# test_labels = []
# for idx,window in enumerate(train_windows_list):
#     if idx % 240 <168 :
#         train.append(windows_datasets[idx][0])
#         train_labels.append(windows_datasets.set_description.labels[idx])
#     else:
#         test.append(windows_datasets[idx][0])
#         test_labels.append(windows_datasets.set_description.labels[idx])

In [ ]:
type(train_subjects_windows_datasets_train_list)

list

In [ ]:
# print(len(train_subjects_windows_datasets_train_list))
# train_subjects_windows_datasets_train_list[12]

### Conversion from list to numpy array

In [ ]:
train_subjects_windows_datasets_train_array = np.array(train_subjects_windows_datasets_train_list)
train_subjects_windows_datasets_test_array = np.array(train_subjects_windows_datasets_test_list)
train_subjects_windows_datasets_val_array = np.array(train_subjects_windows_datasets_val_list)
test_subjects_windows_datasets_all_array = np.array(test_subjects_windows_datasets_all_list)


In [ ]:
train_subjects_train_labels_array = np.array(train_subjects_train_labels)
train_subjects_test_labels_array = np.array(train_subjects_test_labels)
train_subjects_val_labels_array = np.array(train_subjects_val_labels)
test_subjects_labels_array = np.array(test_subjects_labels)

In [ ]:
print(train_subjects_windows_datasets_train_array.shape)
print(train_subjects_train_labels_array.shape)

(1600, 12, 1024)
(1600,)


In [ ]:
print(train_subjects_windows_datasets_test_array.shape)
print(train_subjects_test_labels_array.shape)

(800, 12, 1024)
(800,)


In [ ]:
print(train_subjects_windows_datasets_val_array.shape)
print(train_subjects_val_labels_array.shape)

(800, 12, 1024)
(800,)


In [ ]:
# data_array=np.vstack(data_list)
# label_array=np.hstack(label_list)
# group_array=np.hstack(groups_list)

In [ ]:
# data_array=np.moveaxis(train_array,1,2)
# data_array.shape

In [ ]:
# n_chans = train_windows_datasets[0][0].shape[0]
# input_window_samples = train_windows_datasets[0][0].shape[1]
# print(n_chans)
# print(input_window_samples)

In [ ]:
n_chans = train_subjects_windows_datasets_train_array[0].shape[0]
input_window_samples = train_subjects_windows_datasets_train_array[0].shape[1]
print(n_chans)
print(input_window_samples)

12
1024


In [ ]:
# train_windows_subjects_array

## Data Naming

In [ ]:
X_train = train_subjects_windows_datasets_train_array
y_train = train_subjects_train_labels_array

X_test = train_subjects_windows_datasets_test_array
y_test = train_subjects_test_labels_array

X_val = train_subjects_windows_datasets_val_array
y_val = train_subjects_val_labels_array

external_X = test_subjects_windows_datasets_all_array
external_y = test_subjects_labels_array

# Deep Learning with Braindecode

## Model Configuration

In [ ]:
import tensorflow as tf

# Check if GPU is available
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

# # Print GPU specifications
# !nvidia-smi

Default GPU Device: /device:GPU:0


In [ ]:
# Creating a model
import torch
from braindecode.util import set_random_seeds
# from braindecode.models import ShallowFBCSPNet, EEGNetv4
from braindecode.models import EEGInceptionMI

# Training time
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
import skorch

In [ ]:
cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)
cuda

True

In [ ]:
n_chans = X_train[0].shape[0]
input_window_samples = X_train[0].shape[1]
n_classes = 2
print(n_chans)
print(input_window_samples)

12
1024


In [ ]:
# model = EEGNetv4(
#     n_chans,
#     n_classes,
#     input_window_samples = window_size, #input_window_samples,
#     final_conv_length='auto',
# )
model = EEGInceptionMI(
    12,
    2,
    input_window_seconds = 2,
    sfreq  = 512
)

# Send model to GPU
if cuda:
    model.cuda()

In [ ]:
# model.initial_inception_module

In [ ]:
len(X_train)

1600

In [ ]:
# lr =  0.01 #0.02
lr = 0.00625
weight_decay = 0.5 * 0.001
batch_size = 32 #64 #32
n_epochs = 25 #50 #25 #20 #use few epochs for quick verification

In [ ]:
from skorch.dataset import Dataset
val_dataset = Dataset(X_val, y_val)

In [ ]:
clf =  EEGClassifier(
                    model,
                    criterion=torch.nn.NLLLoss,
                    optimizer=torch.optim.AdamW,
                    # train_split = None,
                    train_split=predefined_split(val_dataset),
                    optimizer__lr=lr,
                    optimizer__weight_decay=weight_decay,
                    batch_size=batch_size,
                    callbacks=[
                        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                    device=device,
                    classes=2,
                    max_epochs=n_epochs,
                    )

In [ ]:
# EEGClassifier

In [ ]:
# clf2.module.state_dict()

## Model Training

In [1]:
clf.fit(X_train, y=y_train)

NameError: name 'clf' is not defined

In [ ]:
internal_test_acc = clf.score(X_test, y=y_test)
print(f"Internal Test acc: {(internal_test_acc * 100):.2f}%")

external_test_acc = clf.score(external_X, y=external_y)
print(f"External Test acc: {(external_test_acc * 100):.2f}%")

Internal Test acc: 65.25%
External Test acc: 65.75%


In [ ]:
clf.fit(X_train, y=y_train , epochs=15)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr       dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  --------
      1            0.9819        0.0905       0.6200            0.6200        2.7985  0.0063  100.6093
      2            0.9938        0.0543       0.7113            0.7113        1.5454  0.0062  100.5726
      3            0.6388        0.0506       0.5150            0.5150        9.9511  0.0061  100.3147
      4            0.8744        0.0839       0.5625            0.5625        5.8791  0.0060  100.4835
      5            0.9956        0.0701       0.6813            0.6813        2.5018  0.0058  100.4981
      6            0.8019        0.0546       0.6438            0.6438        1.9908  0.0056  100.3254
      7            0.9981        0.0266       0.6538            0.6538        3.1811  0.0053  100.

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=================================================================================================================================================================
  Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
  EEGInceptionMI (EEGInceptionMI)                              [1, 12, 1024]             [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1                                 [1, 12, 1024]             [1, 12, 1024, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2                                [1, 12, 1024, 1]          [1, 12, 1, 1024]          --                        --
  ├─_ResidualModuleMI (residual_block_1): 1-3                  [1, 12, 1, 1024]          [1, 288, 1, 1024]         --                        --
  │    └─Conv2d (conv): 2-1           

In [ ]:
internal_test_acc = clf.score(X_test, y=y_test)
print(f"Internal Test acc: {(internal_test_acc * 100):.2f}%")

external_test_acc = clf.score(external_X, y=external_y)
print(f"External Test acc: {(external_test_acc * 100):.2f}%")

Internal Test acc: 63.88%
External Test acc: 70.00%


In [ ]:
clf.fit(X_train, y=y_train , epochs=25);

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr       dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  --------
      1            0.8819        0.0220       0.5625            0.5625        7.5148  0.0063  100.2362
      2            0.9494        0.0778       0.6150            0.6150        5.2012  0.0062  100.3394
      3            0.9894        0.0643       0.7188            0.7188        1.2391  0.0061  100.2108
      4            0.9838        0.0836       0.6438            0.6438        2.8650  0.0060  100.4633
      5            0.9975        0.0235       0.6787            0.6787        2.1087  0.0058  100.2668
      6            0.9912        0.0142       0.7050            0.7050        1.5131  0.0056  100.2795
      7            0.9981        0.0146       0.6750            0.6750        2.5521  0.0053  100.

In [ ]:
internal_test_acc = clf.score(X_test, y=y_test)
print(f"Internal Test acc: {(internal_test_acc * 100):.2f}%")

external_test_acc = clf.score(external_X, y=external_y)
print(f"External Test acc: {(external_test_acc * 100):.2f}%")

In [ ]:
clf2.fit(train_windows_subjects_array, y=train_windows_subjects_labels_array, epochs=10)

  epoch    train_accuracy    train_loss      lr       dur
-------  ----------------  ------------  ------  --------
      1            0.5135        0.7442  0.0100  121.8612
      2            0.6005        0.6132  0.0097  98.5431
      3            0.7401        0.5392  0.0088  98.5580
      4            0.6151        0.4573  0.0075  98.5717
      5            0.6224        0.3844  0.0059  98.4914
      6            0.8661        0.3036  0.0041  98.4945
      7            0.9724        0.1613  0.0025  98.5593
      8            0.9922        0.0673  0.0012  98.4696
      9            0.9922        0.0385  0.0003  98.4445
     10            0.9922        0.0372  0.0000  98.4610


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=================================================================================================================================================================
  Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
  EEGInceptionMI (EEGInceptionMI)                              [1, 12, 1024]             [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1                                 [1, 12, 1024]             [1, 12, 1024, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2                                [1, 12, 1024, 1]          [1, 12, 1, 1024]          --                        --
  ├─_ResidualModuleMI (residual_block_1): 1-3                  [1, 12, 1, 1024]          [1, 288, 1, 1024]         --                        --
  │    └─Conv2d (conv): 2-1           

In [ ]:
test_acc = clf2.score(test_windows_subjects_array, y=test_windows_subjects_labels_array)
print(f"Test acc: {(test_acc * 100):.2f}%")

Test acc: 74.58%


In [ ]:
test_acc = clf2.score(train_windows_subjects_array, y=train_windows_subjects_labels_array)
print(f"Test acc: {(test_acc * 100):.2f}%")

Test acc: 99.22%


In [ ]:
clf2.predict(train_windows_subjects_array)

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
clf2.predict(train_windows_subjects_array[0:1])

array([1])

In [ ]:
best_validation_acc = clf.callbacks_[4][1].best_score_ # a hack to get best validation accuracy
best_validation_acc

0.9129464285714286

In [ ]:
test_acc = clf.score(test_array, y=test_labels_array)
print(f"Test acc: {(test_acc * 100):.2f}%")

Test acc: 69.10%


In [ ]:
# predict for in group
clf.predict(test_array[0:1])

array([1])

In [ ]:
prediction = clf.predict(test_array[0].reshape((1, test_array[0].shape[0], test_array[0].shape[1])))
prediction

array([1])

In [ ]:
print(prediction[0])

1


In [ ]:
test_array[0]

array([[1342603.5, 1477869. , 1473500.8, ..., 2055751.6, 1734616.1,
        1538998.2],
       [1333647.8, 1870885.9, 2351053.5, ..., 2207240.5, 1803791.8,
        1522829.1],
       [1161207.2, 1926396.9, 2676909.2, ..., 1069863. ,  750291. ,
         560874.3],
       ...,
       [ 821411.2, 2028918.1, 3360635.2, ..., 2640455.2, 3471317. ,
        4220229. ],
       [3050925.2, 3810847. , 4501331. , ..., 2354436.5, 3037881.2,
        3633016.8],
       [1449189.1, 3073909.5, 4748868.5, ..., 6759668. , 7629128. ,
        8042702.5]], dtype=float32)

In [ ]:
test_array[0:1].shape

(1, 12, 1024)

In [ ]:
test_array[0].shape[0]

12

In [ ]:
new_test_shape = test_array[0].reshape((1, 12, 1024))
new_test_shape.shape

(1, 12, 1024)

In [ ]:
new_test_shape = test_array[0].reshape((1, test_array[0].shape[0], test_array[0].shape[1]))
new_test_shape.shape

(1, 12, 1024)

In [ ]:
new_test_shape

array([[[1342603.5, 1477869. , 1473500.8, ..., 2055751.6, 1734616.1,
         1538998.2],
        [1333647.8, 1870885.9, 2351053.5, ..., 2207240.5, 1803791.8,
         1522829.1],
        [1161207.2, 1926396.9, 2676909.2, ..., 1069863. ,  750291. ,
          560874.3],
        ...,
        [ 821411.2, 2028918.1, 3360635.2, ..., 2640455.2, 3471317. ,
         4220229. ],
        [3050925.2, 3810847. , 4501331. , ..., 2354436.5, 3037881.2,
         3633016.8],
        [1449189.1, 3073909.5, 4748868.5, ..., 6759668. , 7629128. ,
         8042702.5]]], dtype=float32)

In [ ]:
test_labels_array[0]

1

In [ ]:
clf.

In [ ]:
test_array[0].shape

(12, 1024)

## Data Splitting

###Train Test split based on groups

In [ ]:
import numpy as np

In [ ]:
type(group_list)

list

In [ ]:
X = train_windows_subjects_array  # Features
y = train_windows_subjects_labels_array # Target variable
groups = np.array(group_list) # Group identifiers

In [ ]:
# Proportion of data to allocate to the train set
train_proportion = 0.8

# Initialize lists to store train and test indices
train_indices = []
test_indices = []

In [ ]:
# Get unique group identifiers
unique_groups = np.unique(groups)
unique_groups

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [ ]:
# Iterate over each group
for group_id in unique_groups:
    # Get indices corresponding to the current group
    group_indices = np.where(groups == group_id)[0]

    # Shuffle the indices
    np.random.shuffle(group_indices)

    # Calculate the number of samples for training and testing
    num_samples = len(group_indices)
    num_train_samples = int(train_proportion * num_samples)

    # Split the indices into train and test
    train_indices.extend(group_indices[:num_train_samples])
    test_indices.extend(group_indices[num_train_samples:])

In [ ]:
# Convert indices to arrays
train_indices = np.array(train_indices)
test_indices = np.array(test_indices)

# Split the data
X_train, y_train, groups_train = X[train_indices], y[train_indices], groups[train_indices]
X_test, y_test, groups_test = X[test_indices], y[test_indices], groups[test_indices]

# Check the distribution of groups in train and test sets
train_group_distribution = np.bincount(groups_train.astype(int))
test_group_distribution = np.bincount(groups_test.astype(int))

print("Train Group Distribution:", train_group_distribution)
print("Test Group Distribution:", test_group_distribution)


Train Group Distribution: [  0 192 192 192 192 192 192 192 192]
Test Group Distribution: [ 0 48 48 48 48 48 48 48 48]


In [ ]:
# X_train
# y_train
# X_test
# y_test

In [ ]:
print(len(X_train))
print(len(X_test))

1536
384


###Train Validation Test split based on groups

In [ ]:
# Define the proportions for train, validation, and test sets
train_proportion = 0.6
validation_proportion = 0.2
test_proportion = 0.2

# Initialize lists to store train, validation, and test indices
train_indices = []
validation_indices = []
test_indices = []

# Get unique group identifiers
unique_groups = np.unique(groups)

In [ ]:
# Iterate over each group
for group_id in unique_groups:
    # Get indices corresponding to the current group
    group_indices = np.where(groups == group_id)[0]

    # Shuffle the indices
    np.random.shuffle(group_indices)

    # Calculate the number of samples for training, validation, and testing
    num_samples = len(group_indices)
    num_train_samples = int(train_proportion * num_samples)
    num_validation_samples = int(validation_proportion * num_samples)
    num_test_samples = num_samples - num_train_samples - num_validation_samples

    # Split the indices into train, validation, and test
    train_indices.extend(group_indices[:num_train_samples])
    validation_indices.extend(group_indices[num_train_samples:num_train_samples + num_validation_samples])
    test_indices.extend(group_indices[num_train_samples + num_validation_samples:])

In [ ]:
# Convert indices to arrays
train_indices = np.array(train_indices)
validation_indices = np.array(validation_indices)
test_indices = np.array(test_indices)

# Split the data
X_train, y_train, groups_train = X[train_indices], y[train_indices], groups[train_indices]
X_validation, y_validation, groups_validation = X[validation_indices], y[validation_indices], groups[validation_indices]
X_test, y_test, groups_test = X[test_indices], y[test_indices], groups[test_indices]

# Check the distribution of groups in train, validation, and test sets
train_group_distribution = np.bincount(groups_train.astype(int))
validation_group_distribution = np.bincount(groups_validation.astype(int))
test_group_distribution = np.bincount(groups_test.astype(int))

print("Train Group Distribution:", train_group_distribution)
print("Validation Group Distribution:", validation_group_distribution)
print("Test Group Distribution:", test_group_distribution)

Train Group Distribution: [  0 144 144 144 144 144 144 144 144]
Validation Group Distribution: [ 0 48 48 48 48 48 48 48 48]
Test Group Distribution: [ 0 48 48 48 48 48 48 48 48]


In [ ]:
# X_train
# y_train
# X_test
# y_test
# X_validation
# y_validation

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(X_validation))

1152
384
384


### Predifined Model Training

In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from skorch.helper import predefined_split, SliceDataset

In [ ]:
from skorch.dataset import Dataset

In [ ]:
val_dataset = Dataset(X_validation, y_validation)

In [ ]:
val_dataset[5]

(array([[ 1124860.4 ,   147201.98,  -949044.5 , ...,  3099007.2 ,
          3580560.8 ,  3646823.5 ],
        [ 1366583.9 ,   371192.12,  -755143.2 , ...,  2586315.5 ,
          2820632.8 ,  2730383.  ],
        [  471416.53,  -580919.6 , -1714347.4 , ...,  2718139.  ,
          3217701.8 ,  3371705.5 ],
        ...,
        [ 2114580.5 ,   742072.25,  -915888.3 , ...,  4385780.5 ,
          5365286.5 ,  5938488.5 ],
        [ 2323709.8 ,   914658.4 ,  -767178.2 , ...,  4112408.8 ,
          5349401.  ,  6070588.5 ],
        [  782885.9 ,  -302354.1 , -1625126.5 , ...,  4463744.  ,
          5180584.  ,  5482431.5 ]], dtype=float32),
 1)

In [ ]:
lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 64
n_epochs = 13

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(val_dataset),
    # train_split= predefined_split,
    # train_split= None,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    device=device,
    classes=n_classes,
    max_epochs=n_epochs,
)

In [ ]:
clf.fit(X_train, y=y_train)

  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  -------
      1            0.9948        0.0374       0.9896            0.9896        0.0505  0.0006  64.5870
      2            0.9766        0.0271       0.9349            0.9349        0.1905  0.0006  66.7039
      3            0.9931        0.0175       0.9661            0.9661        0.1213  0.0006  66.5871
      4            1.0000        0.0060       0.9896            0.9896        0.0617  0.0005  66.6253
      5            0.9965        0.0050       0.9661            0.9661        0.1162  0.0005  66.6262
      6            0.9991        0.0072       0.9844            0.9844        0.0692  0.0004  66.6175
      7            0.9983        0.0047       0.9818            0.9818        0.0774  0.0003  66.5919
      8            1.0000        0.0033       0.9896            0.9896        0.06

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=================================================================================================================================================================
  Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
  EEGInceptionMI (EEGInceptionMI)                              [1, 12, 1024]             [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1                                 [1, 12, 1024]             [1, 12, 1024, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2                                [1, 12, 1024, 1]          [1, 12, 1, 1024]          --                        --
  ├─_ResidualModuleMI (residual_block_1): 1-3                  [1, 12, 1, 1024]          [1, 288, 1, 1024]         --                        --
  │    └─Conv2d (conv): 2-1           

In [ ]:
test_acc = clf.score(X_test, y=y_test)
print(f"Test acc: {(test_acc * 100):.2f}%")

Test acc: 99.22%


In [ ]:
test_acc = clf.score(test_windows_subjects_array, y=test_windows_subjects_labels_array)
print(f"Test acc: {(test_acc * 100):.2f}%")

Test acc: 73.75%


In [ ]:
best_validation_acc = clf.callbacks_[4][1].best_score_ # a hack to get best validation accuracy

In [ ]:
best_validation_acc

1.0

### Validation Split (Manual Sklearn)

In [ ]:
# MLP with manual validation set
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:

# split into 80% for train and 20% for validation
X_train, X_test, y_train, y_test = train_test_split(X_train, Y, test_size=0.33, random_state=seed)
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=10)

In [ ]:
# train , test = cv.split(windows_datasets[],y=windows_datasets.set_description.labels, groups=groups)

In [ ]:
clf =  EEGClassifier(
                    model,
                    criterion=torch.nn.NLLLoss,
                    optimizer=torch.optim.AdamW,
                    # iterator_train = StratifiedShuffleSplit(),#cv.split(epochs, y=labels, groups=groups),
                    # train_split=predefined_split(train_set),  # using valid_set for validation
                    # train_split=predefined_split,  # using valid_set for validation
                    # train_split = cv.split(epochs, y=labels, groups=groups),
                    train_split = None,
#                     train_split = skorch.dataset.ValidSplit(cv=cv.split(windows_datasets,
#                           y=windows_datasets.set_description.labels, groups=groups)),

                    optimizer__lr=lr,
                    optimizer__weight_decay=weight_decay,
                    batch_size=batch_size,
                    callbacks=[
                        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                    device=device,
                    # classes=classes,

                    )

In [ ]:
clf.fit(train_windows_subjects_array, y=train_windows_subjects_labels_array, epochs=10)

In [ ]:
clf.fit

## Model Training

In [ ]:
def training_function(windows_datasets, n_epochs=25):
    print('\n', '#'*25, 'Cross Subject Training:', '#'*25, '\n')
    dataset = windows_datasets
    clf.fit(dataset, y=dataset.set_description.labels, epochs=n_epochs);
    best_validation_acc = clf.callbacks_[4][1].best_score_ # a hack to get best validation accuracy
    best_validation_kappa = (2*best_validation_acc)-1
    print("Best Cross Validation Kappa Score: {:.2f}".format(best_validation_kappa))

In [ ]:
# training_function(windows_datasets, n_epochs=n_epochs);